<a href="https://colab.research.google.com/github/gvalexis/cagv_pcd/blob/main/tarea6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install librosa pywavelets tensorflow

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("mmoreaux/audio-cats-and-dogs")
print("Path to dataset files:", path)

100%|██████████| 97.0M/97.0M [00:01<00:00, 74.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mmoreaux/audio-cats-and-dogs/versions/5


In [ ]:
source_dir = path + '/cats_dogs'

In [ ]:
os.listdir(source_dir+'/test/test/')

['dog_barking_90.wav',
 'dog_barking_45.wav',
 'dog_barking_54.wav',
 'dog_barking_44.wav',
 'dog_barking_112.wav',
 'dog_barking_73.wav',
 'dog_barking_48.wav',
 'dog_barking_19.wav',
 'dog_barking_78.wav',
 'dog_barking_24.wav',
 'dog_barking_82.wav',
 'dog_barking_46.wav',
 'dog_barking_3.wav',
 'dog_barking_64.wav',
 'dog_barking_66.wav',
 'dog_barking_49.wav',
 'dog_barking_12.wav',
 'dog_barking_91.wav',
 'dog_barking_89.wav',
 'dog_barking_34.wav',
 'dog_barking_62.wav',
 'dog_barking_59.wav',
 'dog_barking_7.wav',
 'dog_barking_8.wav',
 'dog_barking_99.wav',
 'dog_barking_9.wav',
 'dog_barking_15.wav',
 'dog_barking_43.wav']

In [ ]:
import librosa
import pywt
import numpy as np
import os
from sklearn.model_selection import train_test_split

def extract_wavelet_features(file_path):
    # Cargar el archivo de audio
    y, sr = librosa.load(file_path, sr=None)
    # Aplicar la Transformada Wavelet
    coeffs = pywt.wavedec(y, 'db1', level=5)
    features = np.hstack(coeffs)

    return features

# Directorios de los archivos de audio
dog_dir = source_dir+'/train/dog'
cat_dir = source_dir+'/train/cat'

# Listas para almacenar las características y etiquetas
X = []
y = []

# Procesar archivos de perros
for file_name in os.listdir(dog_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(dog_dir, file_name)
        features = extract_wavelet_features(file_path)
        X.append(features)
        y.append(0)  # Etiqueta 0 para perros

# Procesar archivos de gatos
for file_name in os.listdir(cat_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(cat_dir, file_name)
        features = extract_wavelet_features(file_path)
        X.append(features)
        y.append(1)  # Etiqueta 1 para gatos


from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, dtype='float32', padding='post')


# Convertir a arrays de numpy
X = np.array(X)
y = np.array(y)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                     │ (None, 128)                 │      36,225,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,233,473 (138.22 MB)

 Trainable params: 36,233,473 (138.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 637ms/step - accuracy: 0.5346 - loss: 0.6833 - val_accuracy: 0.6190 - val_loss: 0.6837
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 569ms/step - accuracy: 0.7296 - loss: 0.8130 - val_accuracy: 0.5952 - val_loss: 0.6773
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 365ms/step - accuracy: 0.7955 - loss: 0.8252 - val_accuracy: 0.5714 - val_loss: 0.6794
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 367ms/step - accuracy: 0.8310 - loss: 0.5110 - val_accuracy: 0.6190 - val_loss: 0.6798
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 375ms/step - accuracy: 0.8842 - loss: 0.4802 - val_accuracy: 0.5952 - val_loss: 0.6773
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 372ms/step - accuracy: 0.8833 - loss: 0.7591 - val_accuracy: 0.5476 - val_loss: 0.6822
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 431ms/step - accuracy: 0.9541 - loss: 0.4522 - val_accuracy: 0.5238 - val_loss: 0.6895
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 371ms/step - accuracy: 0.9666 - loss: 0.1048 - val_accuracy: 0.5238 - val_loss:

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5263 - loss: 0.8690
Loss: 0.852453887462616, Accuracy: 0.523809552192688


In [ ]:
def predict_sound(file_path):
    features = extract_wavelet_features(file_path)
    # Reshape to a 2D array with a single sample
    features = features.reshape(1, -1)

    # Pad the features to match the input shape of the model
    features = pad_sequences(features, maxlen=X_train.shape[1], dtype='float32', padding='post')

    prediction = model.predict(features)
    if prediction > 0.5:
        return 'Gato'
    else:
        return 'Perro'

# Ejemplo de inferencia
new_file_path = source_dir+'/test/test/dog_barking_8.wav'
result = predict_sound(new_file_path)
print(f'El sonido es de un: {result}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
El sonido es de un: Perro
